In [2]:
from keras import backend as K
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import jieba
import gensim
from gensim.models import Word2Vec
import copy
import os

from m import  BOW

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from keras.layers import K, Activation
from keras.engine import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D
K.clear_session()
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28

K.clear_session()
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28


def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale


# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])
        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


In [3]:
embedding_dim = 300
maxlen = 30
EMBEDDING_DIM = 300

In [6]:
# 以训练数据为例
data_path_dir = 'data'
data = pd.read_csv(os.path.join(data_path_dir,'cuishou_intent3.csv'),sep='\t')
data.columns = ['content','label']

data_tmp = data.copy(deep=True)

d_ = {}
for key, value in enumerate(set(data_tmp.label)):
    d_[value] = key
data_tmp['label'] = data_tmp['label'].apply(lambda x : d_.get(x))

y_train = np.array(data_tmp.label.tolist())
# 构造embedding字典
bow = BOW(data_tmp.content.apply(jieba.lcut).tolist(), min_count=1, maxlen=30) # 长度补齐或截断固定长度30

vocab_size = len(bow.word2idx)
word2vec = gensim.models.KeyedVectors.load_word2vec_format('data/ft_wv.txt')

embedding_matrix = np.zeros((vocab_size+1,300))
for key, value in bow.word2idx.items():
    if key in word2vec.vocab: # Word2Vec训练得到的的实例需要word2vec.wv.vocab
        embedding_matrix[value] = word2vec.get_vector(key)
    else:
        embedding_matrix[value] = [0] * embedding_dim

Doc To Number: 100%|██████████| 53850/53850 [00:00<00:00, 151585.78it/s]


In [7]:
# label要处理成one-hot的形式
X_train = copy.deepcopy(bow.doc2num)
y_train = copy.deepcopy(y_train)
y_train = np.array(pd.get_dummies(y_train)).astype(int)

In [8]:
def get_capsule_model():
    input1 = Input(shape=(maxlen,))
    embed_layer = Embedding(vocab_size + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)(input1)
    embed_layer = SpatialDropout1D(rate_drop_dense)(embed_layer)

    x = Bidirectional(
        GRU(gru_len, activation='relu', dropout=dropout_p, recurrent_dropout=dropout_p, return_sequences=True))(
        embed_layer)
    capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,
                      share_weights=True)(x)
    capsule = Flatten()(capsule)
    capsule = Dropout(dropout_p)(capsule)
    output = Dense(21, activation='softmax')(capsule)
    model = Model(inputs=input1, outputs=output)
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
    return model

In [9]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True)
for train_idx, val_idx in skf.split(X_train, np.argmax(y_train,axis=1)):
    pass

In [10]:
X_train_1 = copy.deepcopy(X_train[train_idx])
y_train_1 = copy.deepcopy(y_train[train_idx])
X_val = X_train[val_idx]
y_val = y_train[val_idx]

In [11]:
for i in range(5):
    model = get_capsule_model()
    hist = model.fit(X_train_1, y_train_1, validation_data=(X_val,y_val), batch_size=128, epochs=15)

Train on 43088 samples, validate on 10762 samples
Epoch 1/15
43088/43088 [==============================] - 31s 724us/step - loss: 0.3629 - acc: 0.9141 - val_loss: 0.1397 - val_acc: 0.9622
Epoch 2/15
43088/43088 [==============================] - 30s 699us/step - loss: 0.1120 - acc: 0.9702 - val_loss: 0.0790 - val_acc: 0.9799
Epoch 3/15
43088/43088 [==============================] - 29s 678us/step - loss: 0.0801 - acc: 0.9778 - val_loss: 0.0604 - val_acc: 0.9831
Epoch 4/15
43088/43088 [==============================] - 30s 687us/step - loss: 0.0658 - acc: 0.9822 - val_loss: 0.0499 - val_acc: 0.9868
Epoch 5/15
43088/43088 [==============================] - 29s 680us/step - loss: 0.0548 - acc: 0.9846 - val_loss: 0.0510 - val_acc: 0.9856
Epoch 6/15
43088/43088 [==============================] - 29s 683us/step - loss: 0.0509 - acc: 0.9852 - val_loss: 0.0444 - val_acc: 0.9880
Epoch 7/15
43088/43088 [==============================] - 29s 676us/step - loss: 0.0438 - acc: 0.9875 - val_loss: 0.

In [16]:
print(hist.history)

{'val_loss': [0.12892536918396091, 0.08242203289415508, 0.062457309024714115, 0.05588281770091233, 0.048920187518994754, 0.04596260152175632, 0.04150009806749242, 0.04901023257400545, 0.03982942132305238, 0.03930488572838614, 0.038571269717154015, 0.03469154188439563, 0.03497855218819328, 0.032119513189018914, 0.03098908124210843], 'val_acc': [0.9654339341908178, 0.9756550826762295, 0.9816019327041053, 0.9845753577180433, 0.986898346032336, 0.9870841850735533, 0.9878275413270379, 0.9849470358447855, 0.987734621817506, 0.9884779780709905, 0.9893142538340068, 0.9899646905779594, 0.9895930124512172, 0.9905222077459191, 0.9912655640215573], 'loss': [0.36973854530889044, 0.116115629270457, 0.08031857037218892, 0.06620479237206621, 0.0575846308517912, 0.05027253980026607, 0.04535247760693789, 0.04014299534305493, 0.041774897132469346, 0.03520113558636759, 0.03484118176777036, 0.03092092395553658, 0.028736512534540994, 0.02944773405332449, 0.024414141373650052], 'acc': [0.9104623096696604, 0.